In [1]:
# this is required ONLY in a Jupyter Notebook
# otherwise you'll get a "RuntimeError: Cannot run the event loop while another loop is running"
# omit this import and apply in the real code
import nest_asyncio

from ahbicht.content_evaluation import fc_evaluators
from ahbicht.models.mapping_results import PackageKeyConditionExpressionMapping

nest_asyncio.apply()  # can be omitted outside jupyter

In [2]:
from ahbicht.json_serialization.tree_schema import model_dump_tree
from ahbicht.expressions.expression_resolver import parse_expression_including_unresolved_subexpressions

tree = await parse_expression_including_unresolved_subexpressions(
    "Muss [2] U (([3] O [4]) U [123P])[901] U [555]",
    resolve_packages=False,  # false for now because the PackageResolver is not introduced yet
)  # the expression as you get it from the AHB
print(tree)  # The tree is a data structure that represents the expression

model_dump_tree(tree)

Tree(Token('RULE', 'ahb_expression'), [Tree('single_requirement_indicator_expression', [Token('MODAL_MARK', 'Muss'), Tree('and_composition', [Tree('and_composition', [Tree(Token('RULE', 'condition'), [Token('CONDITION_KEY', '2')]), Tree('then_also_composition', [Tree('and_composition', [Tree('or_composition', [Tree(Token('RULE', 'condition'), [Token('CONDITION_KEY', '3')]), Tree(Token('RULE', 'condition'), [Token('CONDITION_KEY', '4')])]), Tree(Token('RULE', 'package'), [Token('PACKAGE_KEY', '123P')])]), Tree(Token('RULE', 'condition'), [Token('CONDITION_KEY', '901')])])]), Tree(Token('RULE', 'condition'), [Token('CONDITION_KEY', '555')])])])])


{'type': 'ahb_expression',
 'children': [{'token': None,
   'tree': {'type': 'single_requirement_indicator_expression',
    'children': [{'tree': None,
      'token': {'value': 'Muss', 'type': 'MODAL_MARK'}},
     {'token': None,
      'tree': {'type': 'and_composition',
       'children': [{'token': None,
         'tree': {'type': 'and_composition',
          'children': [{'token': None,
            'tree': {'type': 'condition',
             'children': [{'tree': None,
               'token': {'value': '2', 'type': 'CONDITION_KEY'}}]}},
           {'token': None,
            'tree': {'type': 'then_also_composition',
             'children': [{'token': None,
               'tree': {'type': 'and_composition',
                'children': [{'token': None,
                  'tree': {'type': 'or_composition',
                   'children': [{'token': None,
                     'tree': {'type': 'condition',
                      'children': [{'tree': None,
                        'token': {'

In [3]:
# Now, to be honest, this looks pretty lengthy and verbose.
# That's why you can serialize the tree in a more concise way using the so-called "concise serialization".
# Note that the concise json dump is _not_ deserializable (yet). So "concise serialization" is a one way street so far.
from ahbicht.json_serialization.tree_schema import model_dump_tree

model_dump_tree(tree, mode="concise")  # this contains all the relevant information and is pretty concise

{'ahb_expression': [{'single_requirement_indicator_expression': ['Muss',
    {'and_composition': [{'and_composition': ['2',
        {'then_also_composition': [{'and_composition': [{'or_composition': ['3',
              '4']},
            '123P']},
          '901']}]},
      '555']}]}]}

In [4]:
# As written in the README, ahbicht will _not_ do any content evaluation for you.
# You'll need to write your own Evaluator classes for:
# 1. Requirement Constraints (RC) for conditions which are fulfilled/unfulfilled/unknown
# 2. Format Constraints (FC) which describe that data must obey a specified format, e.g. number of post decimal places, MaLo-ID etc.
# 3. Hints: "Hinweise", plain text which is passed through
# 4. Packages: From 2022-10-01 it is possible to abbreviate frequently used expressions in "packages".

In [5]:
from ahbicht.content_evaluation.rc_evaluators import RcEvaluator
from maus.edifact import EdifactFormat, EdifactFormatVersion
from ahbicht.content_evaluation.evaluationdatatypes import EvaluationContext, EvaluatableData


class MyRequirementConstraintEvaluator(RcEvaluator):
    """
    A Requirement Constraint (RC) Evaluator answers questions of the kind: "Is the condition <condition_number> fulfilled?"
    Any requirement constraint evaluator has to inherit from RcEvaluator.
    """

    def __init__(self):
        super().__init__()

    def _get_default_context(self) -> EvaluationContext:
        # implement a default context here
        return EvaluationContext(scope=None)

    # in the AHBs the conditions are unique per EDIFACT format / Message Implementation Guide
    # these attributes may be used to distinguish different Evaluator instances and choose the right one
    edifact_format = EdifactFormat.UTILMD
    edifact_format_version = EdifactFormatVersion.FV2104  # valid since 2021-04-01

    # note that requirement constraint evaluation methods can also be async, if necessary.

    def evaluate_2(self, evaluatable_data: EvaluatableData, context: EvaluationContext):
        # Insert your own logic for condition "[2]" here.
        # This can be an async method, which allows you to do e.g. requests to microservices or databases.
        # The outcome of all these methods is always a so called `ConditionFulfilledValue`
        # See the docs: https://ahbicht.readthedocs.io/en/latest/api/ahbicht.expressions.html?highlight=ConditionFulfilledValue#ahbicht.expressions.condition_nodes.ConditionFulfilledValue
        return evaluatable_data.body["2"]

    def evaluate_3(self, evaluatable_data: EvaluatableData, context: EvaluationContext):
        # Insert you own logic for condition "[3]" here,
        # You can also use sync methods for evaluations that don't require any lookups.
        return evaluatable_data.body["3"]

    def evaluate_4(self, evaluatable_data: EvaluatableData, context: EvaluationContext):
        # Insert your own logic for condition "[4]" here
        return evaluatable_data.body["4"]

    # [8] and [9] are part of the package "[123P]"
    def evaluate_8(self, _, __):
        return ConditionFulfilledValue.FULFILLED

    def evaluate_9(self, _, __):
        return ConditionFulfilledValue.FULFILLED

In [6]:
# now we do the same thing for the format constraints
from ahbicht.models.condition_nodes import ConditionFulfilledValue, EvaluatedFormatConstraint
from ahbicht.content_evaluation.fc_evaluators import FcEvaluator


class MyFormatConstraintEvaluator(FcEvaluator):
    """
    Format Constraint (FC) Evaluator answers questions of the kind: "Does the data provided obey format specified in condition <condition_number>?"
    Any format constraint evaluator has to inherit from FcEvaluator.
    """

    edifact_format = EdifactFormat.UTILMD
    edifact_format_version = EdifactFormatVersion.FV2104  # valid since 2021-04-01

    # note that format constraint evaluation methods can also be async, if necessary
    def evaluate_901(self, entered_input):
        """
        This method checks if the entered_input fulfills the constraints of condition 901.
        This could be f.e. if entered_input is a valid OBIS, a valid Marktlokations-ID etc.
        For this MWE we check if the input is all lower case.
        The methods can be either sync or async.
        The result is always an EvaluatedFormatConstraint.
        """
        # insert your own logic here
        if not entered_input:
            return EvaluatedFormatConstraint(
                format_constraint_fulfilled=False, error_message=f"The input is empty but expected lower case."
            )
        if entered_input.lower() == entered_input:
            return EvaluatedFormatConstraint(format_constraint_fulfilled=True)
        return EvaluatedFormatConstraint(
            format_constraint_fulfilled=False,
            error_message=f"The input '{entered_input['data']}' does not obey format constraint 901.",
        )

In [7]:
from ahbicht.expressions.package_expansion import PackageResolver

# we also define a hints provider
from ahbicht.expressions.hints_provider import HintsProvider


class MyHintsProvider(HintsProvider):
    """
    A Hints Provider provides plain text for given "condition" numbers.
    """

    edifact_format = EdifactFormat.UTILMD
    edifact_format_version = EdifactFormatVersion.FV2104

    # note that hint text provisioning methods can also be async, if necessary

    def get_hint_text(self, condition_key: str):
        # Implement your own logic here, e.g. read the actual hint from a database or microservice
        # (spoiler: Hochfrequenz has a public REST API that does exactly that, just drop us a line)
        if condition_key == "555":
            return "Hinweis 555 applies."
        return None


# and a package resolver
class MyPackageResolver(PackageResolver):
    """
    A PackageResolver answers the question: "For which expression is [xxxP] the abbreviation?"
    """

    edifact_format = EdifactFormat.UTILMD
    edifact_format_version = EdifactFormatVersion.FV2104

    async def get_condition_expression(self, package_key: str) -> PackageKeyConditionExpressionMapping:
        if package_key == "123P":
            return PackageKeyConditionExpressionMapping(
                package_key="123P", package_expression="[8] U [9]", edifact_format=EdifactFormat.UTILMD
            )
        return PackageKeyConditionExpressionMapping(
            package_key=package_key, package_expression=None, edifact_format=EdifactFormat.UTILMD
        )

In [8]:
# we'll just provide some hard coded data here for demonstration purposes
hardcoded_content_evaluations = {
    # These are the data that our dummy RC evaluator is going to return.
    # The important thing is, that the RC evaluator has access to the message representation/data, whatever they look like.
    "2": ConditionFulfilledValue.FULFILLED,
    "3": ConditionFulfilledValue.UNFULFILLED,
    "4": ConditionFulfilledValue.FULFILLED,
}

my_evaluatable_data = EvaluatableData(
    body=hardcoded_content_evaluations,
    edifact_format=EdifactFormat.UTILMD,  # format and format version may be used to select the appropriate evaluator instance
    edifact_format_version=EdifactFormatVersion.FV2104,
)  # this is the data that, in real life, contains the content of the edifact message.

In [9]:
from ahbicht.content_evaluation.token_logic_provider import SingletonTokenLogicProvider, TokenLogicProvider
import inject
from ahbicht.content_evaluation.evaluationdatatypes import EvaluatableDataProvider
from ahbicht.expressions.ahb_expression_evaluation import evaluate_ahb_expression_tree


# We use dependency injection to get the evaluators in place:
def provide_evaluatable_data():
    # place your own dynamic(!) data provision method here
    # in web applications it's recommended to access a context variable here that is set in the request handler
    return EvaluatableData(
        body=hardcoded_content_evaluations,
        edifact_format=EdifactFormat.UTILMD,
        edifact_format_version=EdifactFormatVersion.FV2104,
    )


# a token logic provider is a wrapper around all evaluator instance singletons
my_logic_provider = SingletonTokenLogicProvider(
    [MyPackageResolver(), MyHintsProvider(), MyRequirementConstraintEvaluator(), MyFormatConstraintEvaluator()]
)
inject.clear_and_configure(
    lambda binder: binder.bind(TokenLogicProvider, my_logic_provider)
    # While the evaluators are injected as singleton style instances, the evaluatable data may change in each call.
    # This is why they are injected with "bind_to_provider" instead of a simple "bind".
    .bind_to_provider(EvaluatableDataProvider, provide_evaluatable_data)
)

# now that we injected the package resolver, we can also expand the package 123P in the expression
tree = await parse_expression_including_unresolved_subexpressions(
    "Muss [2] U (([3] O [4]) U [123P])[901] U [555]",
    resolve_packages=True,  # must only be true if a package resolver is injected
)

# But later on we can provide AHBicht with the content evaluation results ...
# Providing the content evaluation results to find out, if a line in the AHB is actually required,
# is called expression evaluation.

# If there's any text or user input associated with the field whose expression/tree we evaluated,
# then we can set a context variable that will be used to e.g. evaluate a format constraint.
# This is relevant if you use AHBicht for validation (does my edifact message obey the AHB?).
# If you use the MAUS data model to model the AHB and message data, this will be set under the hood.
fc_evaluators.text_to_be_evaluated_by_format_constraint.set("it's all lower case. should be fine for 901")
# Now the context variable is set and ready to be used by the FC evaluator.

expression_evaluation_result = await evaluate_ahb_expression_tree(tree)

In [10]:
print(f"It's actually a '{expression_evaluation_result.requirement_indicator}'field.")
if expression_evaluation_result.format_constraint_evaluation_result.format_constraints_fulfilled:
    print(f"The format constraint is fulfilled => Data are ok.")
else:
    print(
        f'The format constraint is _not_ fulfilled: "{expression_evaluation_result.format_constraint_evaluation_result.error_message}"'
    )
print(
    f'Please note the following hint: "{expression_evaluation_result.requirement_constraint_evaluation_result.hints}"'
)

It's actually a 'MUSS'field.
The format constraint is fulfilled => Data are ok.
Please note the following hint: "Hinweis 555 applies."


In [11]:
# If writing the RcEvaluator+FcEvaluator+HintsProvider+PackageResolver feels like over-engineering for your use case, we have you covered.
# If you already know the values of the single requirement constraints, format constraints and texts for the hint, you can just generate the Evaluator classes on the fly based on the information you have.
from ahbicht.content_evaluation.evaluator_factory import create_and_inject_hardcoded_evaluators
from ahbicht.models.content_evaluation_result import ContentEvaluationResult

inject.clear()
# If you're only dealing with hard coded content evaluation results, there is a shortcut to evaluate an expression:
# 1. put the hard coded results into a so-called content evaluation result:
content_evaluation_result = ContentEvaluationResult(
    hints={"555": "foo"},
    format_constraints={
        "901": EvaluatedFormatConstraint(format_constraint_fulfilled=True, error_message=None),
    },
    requirement_constraints={
        "2": ConditionFulfilledValue.FULFILLED,
        "3": ConditionFulfilledValue.UNFULFILLED,
        "4": ConditionFulfilledValue.FULFILLED,
        "8": ConditionFulfilledValue.FULFILLED,
        "9": ConditionFulfilledValue.FULFILLED,
    },
)
create_and_inject_hardcoded_evaluators(
    content_evaluation_result,
    evaluatable_data_provider=provide_evaluatable_data,
    edifact_format=EdifactFormat.UTILMD,
    edifact_format_version=EdifactFormatVersion.FV2104,
)  # this does all the magic, no need to manually define classes
evaluated = await evaluate_ahb_expression_tree(tree)
print(evaluated)

requirement_indicator=<ModalMark.MUSS: 'MUSS'> requirement_constraint_evaluation_result=RequirementConstraintEvaluationResult(requirement_constraints_fulfilled=True, requirement_is_conditional=True, format_constraints_expression='[901]', hints='foo') format_constraint_evaluation_result=FormatConstraintEvaluationResult(format_constraints_fulfilled=True, error_message=None)


In [12]:
# Now, if your main application is not written in Python and you cannot/must not host AHBicht anywhere or don't want to use our public AHBicht REST API:
# There is a simple way to pre-calculate all possible outcomes of an expression in advance:
from ahbicht.expressions.condition_expression_parser import extract_categorized_keys

categorized_key_extract = await extract_categorized_keys("Muss [2] U ([3] O [4])[901] U [555]")
print(categorized_key_extract)

hint_keys=['555'] format_constraint_keys=['901'] requirement_constraint_keys=['2', '3', '4'] package_keys=[] time_condition_keys=[]


In [13]:
precalculated_results = categorized_key_extract.generate_possible_content_evaluation_results()
print(len(precalculated_results))  # <-- this contains 128 possible ContentEvaluationResults

54


In [14]:
results = list()
for content_evaluation_result in precalculated_results:
    create_and_inject_hardcoded_evaluators(
        content_evaluation_result,
        evaluatable_data_provider=provide_evaluatable_data,
        edifact_format=EdifactFormat.UTILMD,
        edifact_format_version=EdifactFormatVersion.FV2104,
    )
    try:
        expression_evaluation_result = await evaluate_ahb_expression_tree(
            await parse_expression_including_unresolved_subexpressions(expression="Muss [2] U ([3] O [4])[901] U [555]")
        )
    except NotImplementedError:
        # there are cases that don't make any sense and won't occur out in the wild. These are mostly related to neutral elements where no neutral elements are expected. We can just ignore them
        continue
    results.append((content_evaluation_result, expression_evaluation_result))
    # export the result with expression_evaluation_result and the content_evaluation_result and deserialize them in your non-python application
print(results[0:2])

[(ContentEvaluationResult(hints={'555': 'Hinweis 555'}, format_constraints={'901': EvaluatedFormatConstraint(format_constraint_fulfilled=True, error_message=None)}, requirement_constraints={'2': <ConditionFulfilledValue.FULFILLED: 'FULFILLED'>, '3': <ConditionFulfilledValue.FULFILLED: 'FULFILLED'>, '4': <ConditionFulfilledValue.FULFILLED: 'FULFILLED'>}, packages={}, id=None), AhbExpressionEvaluationResult(requirement_indicator=<ModalMark.MUSS: 'MUSS'>, requirement_constraint_evaluation_result=RequirementConstraintEvaluationResult(requirement_constraints_fulfilled=True, requirement_is_conditional=True, format_constraints_expression='[901]', hints='foo'), format_constraint_evaluation_result=FormatConstraintEvaluationResult(format_constraints_fulfilled=True, error_message=None))), (ContentEvaluationResult(hints={'555': 'Hinweis 555'}, format_constraints={'901': EvaluatedFormatConstraint(format_constraint_fulfilled=True, error_message=None)}, requirement_constraints={'2': <ConditionFulfill